our typical pytorch pipeline consists of three steps
1) Design model (input,output size,forward pass)
2) construct loss and optimizer
3) training loop


-- forward pass : comopute prediction

-- backward pass: gradients calculations

-- update weights

 from training loop  to last we do cople of times untill we are done

In [243]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [244]:
# 0) Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


In [245]:
#scale for getting 0 mean and unique std
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [246]:
#converting into tensors

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [247]:
# 1) model
# Linear model f = wx + b , sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1) #define our layer here one layer

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegression(n_features)

In [248]:
# 2) Loss and optimizer
num_epochs = 400
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [249]:
# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 40 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 40, loss = 0.3034
epoch: 80, loss = 0.2366
epoch: 120, loss = 0.2020
epoch: 160, loss = 0.1801
epoch: 200, loss = 0.1646
epoch: 240, loss = 0.1529
epoch: 280, loss = 0.1435
epoch: 320, loss = 0.1359
epoch: 360, loss = 0.1295
epoch: 400, loss = 0.1241


In [250]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round() # if val> .5 then 1 and 0 otherwise
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9474


evaluaion should not be recorded by pytorch. in other words required_grad needs to be false in this case .That's why we used torch.no_grad()